# ROS2 bag file converter

## Imports and variables

In [1]:
import os
from pathlib import Path

from rosbags.dataframe import get_dataframe
from rosbags.highlevel import AnyReader
from rosbags.typesys import get_types_from_msg, register_types

workspace_dir='/home/hugobaptista/ros2/ccpm'
bag_dir='IMU-rosbag_11032024'

# Add all topic message types to the reader's registered types
add_types = {}
for filename in os.listdir(f'{workspace_dir}/src/ccpm_msgs/msg'):
    text = Path(f'{workspace_dir}/src/ccpm_msgs/msg/{filename}').read_text()
    name=f'ccpm_msgs/msg/{filename.split(".")[0]}'
    add_types.update(get_types_from_msg(text, name))
register_types(add_types)

## Convert from bag to Pandas dataframe

In [2]:
def convert_index_to_timestamp_column(df):
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'timestamp'}, inplace=True)

topics={}
dataframes={}
with AnyReader([Path(f'{workspace_dir}/{bag_dir}')]) as reader:
    # The 'topics' dictionary's keys are the topics names and the values are their arguments (list)
    for connection in reader.connections:
        keys = []
        for field in reader.typestore.FIELDDEFS[connection.msgtype][1]:
            keys.append(field[0])
        topics[connection.topic]=keys

    # The 'dataframes' dictionary's keys are the topics names and the values are their pandas dataframe
    for topic in topics:
        dataframes[topic]=get_dataframe(reader, topic, topics.get(topic))
        convert_index_to_timestamp_column(dataframes[topic])

for topic in dataframes:
    print(f'Topic: {topic}\nDataframe:{dataframes[topic].head}\n-----------------------------------------\n')

/motor0/status message #39559 <class 'AssertionError'> error: 
/motor0/status message #104212 <class 'AssertionError'> error: 
/motor0/status message #110885 <class 'AssertionError'> error: 
/motor0/status message #114210 <class 'struct.error'> error: unpack_from requires a buffer of at least 77 bytes for unpacking 1 bytes at offset 76 (actual buffer size is 76)
/motor0/status message #134537 <class 'struct.error'> error: unpack_from requires a buffer of at least 77 bytes for unpacking 1 bytes at offset 76 (actual buffer size is 76)
/motor0/status message #137086 <class 'struct.error'> error: unpack_from requires a buffer of at least 77 bytes for unpacking 1 bytes at offset 76 (actual buffer size is 76)
/motor0/status message #150811 <class 'struct.error'> error: unpack_from requires a buffer of at least 77 bytes for unpacking 1 bytes at offset 76 (actual buffer size is 76)
/motor0/status message #175749 <class 'AssertionError'> error: 
/motor0/status message #184391 <class 'AssertionE

## Save the Pandas dataframes as csv files

In [3]:
for topic in dataframes:
    new_file=f'{workspace_dir}/{bag_dir}/csv/{topic[1:].replace("/", "_")}.csv'
    dataframes[topic].to_csv(new_file, index=False)
    print(f'The topic {topic} was saved as {new_file}')

The topic /motor0/status was saved as /home/hugobaptista/ros2/ccpm/IMU-rosbag_11032024/csv/motor0_status.csv
The topic /gps/receive was saved as /home/hugobaptista/ros2/ccpm/IMU-rosbag_11032024/csv/gps_receive.csv
The topic /can_gpio/key_switch was saved as /home/hugobaptista/ros2/ccpm/IMU-rosbag_11032024/csv/can_gpio_key_switch.csv
The topic /IMU/data was saved as /home/hugobaptista/ros2/ccpm/IMU-rosbag_11032024/csv/IMU_data.csv
